## ENGG*6600: Special Topics in Information Retrieval - Fall 2022
##Assignment 6: Clustering (Total : 100 points)

**Description**

This is a coding assignment where you will implement the k-means clustering algorithm to cluster passages. Basic proficiency in Python is recommended.  

**Instructions**

* To start working on the assignment, you would first need to save the notebook to your local Google Drive. For this purpose, you can click on *Copy to Drive* button. You can alternatively click the *Share* button located at the top right corner and click on *Copy Link* under *Get Link* to get a link and copy this notebook to your Google Drive.  

*   For questions with descriptive answers, please replace the text in the cell which states "Enter your answer here!" with your answer. If you are using mathematical notation in your answers, please define the variables.
*   You should implement all the functions yourself and should not use a library or tool for the computation.
*   For coding questions, you can add code where it says "enter code here" and execute the cell to print the output.
* To create the final pdf submission file, execute *Runtime->RunAll* from the menu to re-execute all the cells and then generate a PDF using *File->Print->Save as PDF*. Make sure that the generated PDF contains all the codes and printed outputs before submission.
To create the final python submission file, click on File->Download .py.


**Submission Details**

* Due data: November. 28, 2022 at 11:59 PM (EDT).
* The final PDF and python file must be uploaded on CourseLink.
* After copying this notebook to your Google Drive, please paste a link to it below. Use the same process given above to generate a link. ***You will not recieve any credit if you don't paste the link!*** Make sure we can access the file.
***LINK: *https://colab.research.google.com/drive/1pU2XaxvD1UhkTdBlq1ecevmwYQ_wvVTp?usp=sharing***

**Academic Honesty**

Please follow the guidelines under the *Collaboration and Help* section in the first lecture.      

# Download input files

Please execute the cell below to download the input files.

In [ ]:

import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


import os
import zipfile
import numpy as np

download = drive.CreateFile({'id': '1NR5jMPJAJHD3flGEbckMr8WerfPtLtZ2'})
download.GetContentFile('HW06.zip')

with zipfile.ZipFile('HW06.zip', 'r') as zip_file:
    zip_file.extractall('./')
os.remove('HW06.zip')
# We will use hw05 as our working directory
os.chdir('HW06')

#Setting the input file
passage = "passages.tok.clean_kstem"
init_centroid_file = "centroid.txt"


# 1 : Initial Data Setup (25 points)

We use collection from the ANTIQUE  [https://arxiv.org/pdf/1905.08957.pdf] dataset for this assignment. As described in the previous assignments, this is a passage retrieval dataset.

The description of the input files provided for this assignment is given below.

**Collection file**

Each row of the file consists of the following information:

*passage_id  passage_text*

The id and text information is tab separated. The passage text has been pre-processed to remove punctutation, tokenised and stemmed using the Krovetz stemmer. The terms in the passage text can be accessed by splitting the text based on space.

**Centroid vector values**

Each row is a tab separated entry where the first column is cluster id and the second column is the vector which is used to initialize the cluster centroid in the k-means Algorithm.

In the cell below, collection information as vocababulary size, document frequency and passage count and initial centroid vector values have been given. This can be used in subsequent cells for computation.



In [ ]:
'''
In this function, load the initial centroid values for k clusters.
Return Variables:
init_centroid_vec - array where each element corresponds to the centroid vector.
'''

# load initial centroid values
def load_centroid_init(init_centroid_file):
  init_centroid_vec = []
  for line in open(init_centroid_file):
   row = line.strip().split('\t')
   init_centroid_vec.append(np.fromstring(row[1][1:-1],sep=',').astype(float))
  return init_centroid_vec


'''
In this function, iterate through the input passage and load the data
Return Variables:
vocab - dict mapping word to an integer [0,len(vocab)]
df - dict mapping word to document frquency
num_passages - total number of passages in the input collection
docs - dict mapping each passage id to unique integer between [0,len(num_passages)]
'''
# load vocabulary
def load_vocab(passage):
  df = {}
  vocab = {}
  count = 0
  doc_count = 0
  num_passages = 0
  docs = {}
  for line in open(passage):
   row = line.strip().split('\t')
   docs[row[0]] = doc_count
   doc_count+=1
   terms = row[1].split(' ')
   num_passages+=1
   for word in set(terms):
    if word not in df:
      df[word]=0
    df[word]+=1
   for word in terms:
    if word not in vocab:
      vocab[word]=count
      count+=1

  return vocab,df,num_passages,docs

init_centroid_vec = load_centroid_init(init_centroid_file)
vocab, df, num_passages, docs = load_vocab(passage)

print('Vocabulary Size : {0}'.format(len(vocab)))
print('Total Number of Passages: {0}'.format(num_passages))


Vocabulary Size : 3630
Total Number of Passages: 500


**Passage Representation**

In order to cluster passages, each passage has to be converted into a vector representation. For this assignment, we choose a $tf{\text -}idf$ representation. Every passage is represented by a vector with dimensionality equal to vocab_size. Each dimension in the vector corresponds to a word. The value of each dimension of the vector is the $tf{\text -}idf$ value of the word in that passage. These vectors can be stacked to create a matrix where each row of the matrix corresponds to a passage.

Let the vocab_size be $n$ and num_passages be $m$.

In the cell below, implement the following:

1) Create an input data matrix $P = m \times n$  where $P_{i,j}=tf{\text -}idf(i,j)$. The definition of the $tf{\text -}idf(i,j)$ formulation is given below.

$$ tf{\text -}idf(i,j) = ln(1+count(i,j)) ln(1 + \frac{|C|}{df(j)}) $$


$tf{\text -}idf(i,j)$: the  $tf{\text -}idf$ value of the word corresponding to integer $j$ and passage corresponding to integer $i$ as defined in vocab and docs data structures, respectively.

$count(i,j)$ = number of times word corresponding to integer $j$ occurs in passage corresponding to integer $i$.

$df(j)$ - The document frequency of word corresponding to integer $j$ (i.e., the number of documents that contain the $j^{th}$ word.

$|C|$ - Total number of passages in the collection.


In [ ]:
'''
In this function, create input matrix
Return Variables:
pass_matrix - matrix of shape m by n where each row corresponds to a passage and each column to a word
'''
passages={}
for line in open(passage,encoding='utf8'):
    pid,psg=line.strip().split('\t')
    passages[pid]=[psg]

vocab_keys=list(vocab.keys())

def create_input_matrix(passage,df,num_passages,vocab):
    #enter code here
    pass_matrix=np.zeros((num_passages,len(vocab)))
    for i,p in enumerate(passages):
        for j,word in enumerate(vocab_keys):
            count=str(passages[p]).lower().split().count(word)
            tf=np.log(1+count)
            x=num_passages/df[word]
            idf=np.log(1+x)
            tfidf=tf*idf
            pass_matrix[i][j]=tfidf

    return pass_matrix

pass_matrix =  create_input_matrix(passage,df,num_passages,vocab)

print('Shape of the input matrix : {0}'.format(np.shape(pass_matrix)))


Shape of the input matrix : (500, 3630)


# 2 :  k-means Algorithm (45 points)

In this section you must implement the k-means clustering algorithm. The K-means function is called with $k=3$ ($k$ is the number of clusters) and the initial centroids have been set for you, the distance metric to be used is the squared Euclidean distance. Execute the algorithm for 30 iterations.


In [ ]:

# centroid is a matrix instantiated with initial centroid values
'''
In this function, centroid is instantiated with initial centroid values
Here we assume for k=3, the cluster ids are {0,1,2}
Return Variables:
centroid - matrix where rows correspond to clusters and columns correspond to words.
'''
def  init_centers(vocab, k, init_centroid_vec):
  vocab_size = len(vocab)
  centroid = np.zeros((k, vocab_size), dtype='float')
  for i in range(len(init_centroid_vec)):
     centroid[i] = init_centroid_vec[i]
  return centroid

centroid = init_centers(vocab, 3, init_centroid_vec)

In [ ]:
'''
In this function, implement kmeans algorithm
Return Variables:
final_cluster_assignment - array with cluster id values indexed by the passageid (integer mappings in docs)
num_passage_cluster_final - dict with cluster-id and key and number of elements in the cluster as value
'''
def kmeans(num_passages, pass_matrix, centroid, k):
  final_cluster_assignment = [0] * num_passages
  num_passage_cluster_final = {}

  for i in range(30):
    for p in range(num_passages):
      init = np.inf
      for c in range(k):
        dist = np.linalg.norm(pass_matrix[p]-centroid[c])

        if(init > dist):
          init = dist;
          final_cluster_assignment[p] = c

    clust = [0] * k
    avg_centroid = [[0]* len(centroid[0])] * k
    for p in range(num_passages):
      cv = final_cluster_assignment[p]
      clust[cv] += 1
      avg_centroid[cv] = np.add(avg_centroid[cv] ,pass_matrix[p])

    for n in range(k):
      centroid[n] = avg_centroid[n]/clust[n]

  for c in range(k):
    num_passage_cluster_final[c] = final_cluster_assignment.count(c)

  return final_cluster_assignment,num_passage_cluster_final



final_cluster_assignment, num_passage_cluster_final = kmeans(num_passages, pass_matrix, centroid, 3)

print('Number of initial centroids : {0}'.format(len(centroid)) )

# print number of elements in each cluster
'''
 Hint: The values lies within the interval [360,370] for the largest cluster,
 [120,130] for the second largest cluster and [0,10] for the smallest cluster.
'''
for cid,cnum in num_passage_cluster_final.items():
   print(cid, cnum)



Number of initial centroids : 3
0 371
1 120
2 9


# 3 :  Evaluation (30 points)

In this section, you must implement two evaluation metrics:

1) IntraCluster Similarity Metric - Average Diameter
Distance of each cluster $S$

$$Sim(S) = \frac{1}{|S| (|S|-1)} \sum_{x,y \in S,x \neq y} dist(x,y)$$

$|S|$ - number of passages assigned to cluster $S$

$dist(x,y)$ - the squared euclidean distance between passages $x$ and $y$.


2) Intercluster Similarity Metric -
Average Linkage Distance between a pair of clusters $S,T$

$$Sim(S,T) = \frac{1}{|S| |T|} \sum_{x \in S,y \in T} dist(x,y)$$

$|S|$ - number of passages assigned to cluster $S$

$|T|$ - number of passages assigned to cluster $T$

$dist(x,y)$ - the squared euclidean distance between passages $x$ and $y$.


You have to calculate and print out the IntraCluster Similarity Metric for every cluster and Intercluster Similarity Metric for every cluster pair. This is illustrated below with examples.

For IntraCluster metric, Average Diameter Distance for cluster (a, b, c),
the value can be calculated as follows :$\frac{dist(a, b)+dist(a, c)+dist(b, a)+dist(b, c)+ dist(c, a)+ dist(c, b)}{3*2} $

For InterCluster metric, Average Linkage Distance for clusters (a, b, c) and (d, e) , the value can be calculated as follows:  $\frac{dist(a, d)+dist(a, e)+ dist(b, d)+ dist(b, e)+ dist (c, d)+ dist(c, e)}{3*2}$



In [ ]:
def average_diameter_dist(num_passage_cluster_final,final_cluster_assignment,pass_matrix):
    sim_score = {}
    k = 3

    for c in range(k):
      clust_list = []
      for idx, value in enumerate(final_cluster_assignment):
        if value == c:
          clust_list.append(idx)

      dist = 0.0
      for i in range(len(clust_list)):
        for j in range(i+1,len(clust_list)):
          dist += np.sum(np.square(pass_matrix[clust_list[i]]-pass_matrix[clust_list[j]]))

      num_p_clust = num_passage_cluster_final[c]
      sim_s = 1/(num_p_clust*(num_p_clust-1))

      sim_score[c] = (sim_s * dist*2)

    return sim_score

'''
In this function, implement intercluster similarity metric Average Linkage Distance
Return Variables:
sim_score - dict with metric score corresponding to each pair of clusters
'''
def average_linkage_dist(num_passage_cluster_final,final_cluster_assignment,pass_matrix):
    sim_score = {}
    k = 3
    p = 0
    for c1 in range(k-1):
      clust_list1 = []

      num_p_clust1 = num_passage_cluster_final[c1]
      for idx, value in enumerate(final_cluster_assignment):
          if value == c1:
            clust_list1.append(idx)

      for c2 in range(c1+1,k):
        clust_list2 = []
        for idx, value in enumerate(final_cluster_assignment):
          if value == c2:
            clust_list2.append(idx)

        eucl_Dist = 0.0
        for i in range(len(clust_list1)):
          for j in range(len(clust_list2)):
            eucl_Dist += np.sum(np.square(pass_matrix[clust_list1[i]]-pass_matrix[clust_list2[j]]))

        num_p_clust2 = num_passage_cluster_final[c2]
        sim_s_t = 1/(num_p_clust1*num_p_clust2)

        sim_score[p] = (sim_s_t * eucl_Dist)
        p = p + 1

    return sim_score

intra_score  = average_diameter_dist(num_passage_cluster_final,final_cluster_assignment,pass_matrix)
inter_score =  average_linkage_dist(num_passage_cluster_final,final_cluster_assignment,pass_matrix)

# print intracluster scores for very cluster
'''
Hint: The value lies within the intervals [310,320] for the largest cluster,
[1135,1145] for the second largest cluster and [3860,3870] for the smallest cluster.
'''
for cid,score in intra_score.items():
   print(cid, score)

# print intercluster scores for every pair of cluster
'''
Hint: The value lies within the intervals [740,750] between largest and second largest clusters.
[2260,2270] between largest and smallest clusters and
[2590,2600] between second largest and smallest cluster
'''
for cid_pair,score in inter_score.items():
   print(cid_pair, score)

0 297.02340250235443
1 1129.4317675677546
2 3829.755796558129
0 728.5627971555085
1 2238.5859474342788
2 2572.1757538320253
